In [1]:
from pyspark.sql import SparkSession, functions as F, types as T

In [2]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

24/02/15 19:23:44 WARN Utils: Your hostname, maksym-HP-Pavilion-Gaming-Laptop-16-a0xxx resolves to a loopback address: 127.0.1.1; using 192.168.0.111 instead (on interface wlo1)
24/02/15 19:23:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/15 19:23:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(spark.version)

3.4.1


In [4]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [5]:
# тут має бути розвʼязок задачі
category_df.join(film_category_df, on=category_df.category_id == film_category_df.category_id, how='left') \
    .select("name").groupBy('name').count().select(F.col('name').alias("category"), F.col('count')).orderBy(F.desc("count")).show()

+-----------+-----+
|   category|count|
+-----------+-----+
|     Sports|   74|
|    Foreign|   73|
|     Family|   69|
|Documentary|   68|
|  Animation|   66|
|     Action|   64|
|        New|   63|
|      Drama|   62|
|      Games|   61|
|     Sci-Fi|   61|
|   Children|   60|
|     Comedy|   58|
|     Travel|   57|
|   Classics|   57|
|     Horror|   56|
|      Music|   51|
+-----------+-----+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [22]:
# тут має бути розвʼязок задачі
rental_df.join(inventory_df, on=inventory_df.inventory_id == rental_df.inventory_id) \
    .join(film_df, on=film_df.film_id == inventory_df.film_id) \
    .join(film_actor_df, on=film_actor_df.film_id == film_df.film_id) \
    .join(actor_df, on=actor_df.actor_id == film_actor_df.actor_id) \
    .groupBy(actor_df.actor_id) \
    .agg(F.first('first_name'), F.first('last_name'), F.count('rental_id').alias('count')) \
    .orderBy(F.desc("count")) \
    .select(F.concat('first(first_name)', F.lit(' '), 'first(last_name)').alias('actor_name'), "count").limit(10).show()


+------------------+-----+
|        actor_name|count|
+------------------+-----+
|    GINA DEGENERES|  753|
|    MATTHEW CARREY|  678|
|       MARY KEITEL|  674|
|ANGELA WITHERSPOON|  654|
|       WALTER TORN|  640|
|       HENRY BERRY|  612|
|       JAYNE NOLTE|  611|
|        VAL BOLGER|  605|
|     SANDRA KILMER|  604|
|      SEAN GUINESS|  599|
+------------------+-----+



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [61]:
# тут має бути розвʼязок задачі
payment_df.join(rental_df, on=payment_df.rental_id == rental_df.rental_id, how='left') \
    .join(inventory_df, on=inventory_df.inventory_id == rental_df.inventory_id, how='left') \
    .join(film_df, on=film_df.film_id == inventory_df.film_id, how='left') \
    .join(film_category_df, on=film_category_df.film_id == film_df.film_id, how='left') \
    .join(category_df, on=category_df.category_id == film_category_df.category_id, how='left') \
    .groupBy(category_df.name).agg(F.sum('amount').alias('rental_sum')).orderBy(F.desc("rental_sum")).limit(1).show()

+------+-----------------+
|  name|       rental_sum|
+------+-----------------+
|Sports|5314.209999999847|
+------+-----------------+



4.
Вивести назви фільмів, яких не має в inventory.

In [51]:
# тут має бути розвʼязок задачі
film_df.join(inventory_df, on= inventory_df.film_id == film_df.film_id, how='left') \
    .select(F.col('title')).filter(inventory_df.film_id.isNull()).show(truncate=False)

+----------------------+
|title                 |
+----------------------+
|ALICE FANTASIA        |
|APOLLO TEEN           |
|ARGONAUTS TOWN        |
|ARK RIDGEMONT         |
|ARSENIC INDEPENDENCE  |
|BOONDOCK BALLROOM     |
|BUTCH PANTHER         |
|CATCH AMISTAD         |
|CHINATOWN GLADIATOR   |
|CHOCOLATE DUCK        |
|COMMANDMENTS EXPRESS  |
|CROSSING DIVORCE      |
|CROWDS TELEMARK       |
|CRYSTAL BREAKING      |
|DAZED PUNK            |
|DELIVERANCE MULHOLLAND|
|FIREHOUSE VIETNAM     |
|FLOATS GARDEN         |
|FRANKENSTEIN STRANGER |
|GLADIATOR WESTWARD    |
+----------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [71]:
# тут має бути розвʼязок задачі
film_df.join(film_actor_df, on=film_actor_df.film_id == film_df.film_id, how='left') \
    .join(film_category_df, on=film_category_df.film_id == film_df.film_id, how='left') \
    .join(actor_df, on=actor_df.actor_id == film_actor_df.actor_id, how='left') \
    .join(category_df, on=category_df.category_id == film_category_df.category_id, how='left') \
    .filter(category_df.name == "Children") \
    .groupBy(actor_df.actor_id, actor_df.first_name, actor_df.last_name).count() \
    .select(F.concat(actor_df.first_name, F.lit(' '), actor_df.last_name).alias('actor_name'), "count") \
    .orderBy(F.desc("count"), actor_df.first_name, actor_df.last_name).limit(3).show()

+-------------+-----+
|   actor_name|count|
+-------------+-----+
| HELEN VOIGHT|    7|
|KEVIN GARLAND|    5|
|   MARY TANDY|    5|
+-------------+-----+



Stop Spark session:

In [72]:
spark.stop()